# Create ACS Index and Azure SQL Database for Avatar Demo
Use this notebook to create an Azure Cognitive Search Index and an Azure SQL Database and populate demo content for the Avatar outdoor shop application.  

Ensure that you have the the Microsoft ODBC driver for SQL Server installed. Here are the instructions for Linux based systems:  
https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver16&tabs=ubuntu18-install%2Calpine17-install%2Cdebian8-install%2Credhat7-13-install%2Crhel7-offline#18


## Setup

In [191]:
# Install the required libraries
%pip install azure-search-documents==11.4.0b6 openai==0.28.1 tenacity pyodbc pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [192]:
import os
import json  
import pandas as pd

import pyodbc
import requests
import inspect

import openai  
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    VectorSearchAlgorithmConfiguration,  
)  

You need to have the following settings for your Azure resources defined in the `local.settings.json` file in the __api__ subfolder to populate the demo content for the outdoor app:

In [193]:
# Load JSON file
with open('./api/local.settings.json', 'r') as file:
    data = json.load(file)

# Azure Cognitive Search
service_endpoint = data["Values"]["AZURE_SEARCH_ENDPOINT"]
key = data["Values"]["AZURE_SEARCH_API_KEY"]
index_name = data["Values"]["AZURE_SEARCH_INDEX"]

# Blob SAS URL for Azure Storage Account
blob_sas_url = data["Values"]["BLOB_SAS_URL"]

# Azure OpenAI
openai.api_type = "azure"
openai.api_key = data["Values"]["AZURE_OPENAI_API_KEY"]
openai.api_base = data["Values"]["AZURE_OPENAI_ENDPOINT"]
openai.api_version = data["Values"]["AZURE_OPENAI_API_VERSION"]
AOAI_embeddings_deployment = data["Values"]["AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT"]

# Azure SQL Database
sql_db_server = data["Values"]["SQL_DB_SERVER"]
sql_db_user = data["Values"]["SQL_DB_USER"]
sql_db_password = data["Values"]["SQL_DB_PASSWORD"]
sql_db_name = data["Values"]["SQL_DB_NAME"]

## Create Cognitive Search Index
First, we create a new Index with demo data to the Cognitive Search service that you have deployed manually.

In [194]:
credential = AzureKeyCredential(key)

df = pd.read_csv('data/citizens-mock-data.csv', dtype={'CID': str})
display(df.head())
input_data = df.to_dict(orient='records')

,CID,FirstName,SecondName,ThirdName,LastName,Nationality,DOB,Governorate,Area,Block,Street,Building,Gender
0,229897660,Ali,Abdullatif,Hussien,Safar,Kuwaiti,22/3/1983,Mubarak Al -Kabeer,Adan,3,23,10,Male
1,229897661,Mohammad,Nasser,Ahamd,Salem,Kuwaiti,19/5/1984,Hawally,Rumithiya,1,45,4,Male
2,229897662,Abdulla,Hussien,Khaled,Yousef,Kuwaiti,21/12/1985,Mubarak Al -Kabeer,Sabah Al Salem,4,67,10,Male
3,229897663,Mariam,Hassan,Fawaz,Ibrahim,Kuwaiti,11/1/1986,Assimah,Mansouriya,12,1,5,Female
4,229897664,Ali,Yasser,Nasser,Alaa,Kuwaiti,17/4/1987,Farwaniya,Khyitan,5,15,12,Male


In [195]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields, also used for query embeddings
def generate_embeddings(text):
    response = openai.Embedding.create(
        input=text, engine=AOAI_embeddings_deployment)
    embeddings = response['data'][0]['embedding']
    return embeddings

In [196]:
# Generate embeddings for title and content fields
for item in input_data:
    firstName = str(item['FirstName'])
    secondName = str(item['SecondName'])
    thirdName = str(item['ThirdName'])
    lastName = str(item['LastName'])

    fullName = firstName + ' ' + secondName + ' ' + thirdName + ' ' + lastName  
    
    governorate = str(item['Governorate'])
    area = str(item['Area'])
    block = str(item['Block'])
    street = str(item['Street'])
    building = str(item['Building'])
    
    address = governorate + ' ' + area + ' ' + block + ' ' + street + ' ' + building

    fullName_embeddings = generate_embeddings(fullName)
    address_embeddings = generate_embeddings(address)

    item['Area'] = area
    item['Block'] = block
    item['Street'] = street
    item['Building'] = building

    item['Name'] = fullName
    item['Address'] = address
    
    item['fullName_vector'] = fullName_embeddings
    item['address_vector'] = address_embeddings


In [197]:
# Output embeddings to docVectors.json file
with open("./data/citizens-mock-data-vectors.json", "w") as f:
    json.dump(input_data, f)

In [198]:
# Delete ACS index if it exists
index_client = SearchIndexClient(endpoint=service_endpoint, credential=credential)

try:
    if index_client.get_index(index_name):
        print('Deleting existing index...')
        index_client.delete_index(index_name)

except:
    print('Index does not exist. No need to delete it.')

Deleting existing index...


In [199]:
# Create a search index
index_client = SearchIndexClient(endpoint=service_endpoint, credential=credential)

fields = [
    SimpleField(name="CID", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="FirstName", type=SearchFieldDataType.String),
    SearchableField(name="SecondName", type=SearchFieldDataType.String),
    SearchableField(name="ThirdName", type=SearchFieldDataType.String),
    SearchableField(name="LastName", type=SearchFieldDataType.String),
    SearchableField(name="Name", type=SearchFieldDataType.String),
    SearchableField(name="Nationality", type=SearchFieldDataType.String),
    SearchableField(name="DOB", type=SearchFieldDataType.String),
    SearchableField(name="Governorate", type=SearchFieldDataType.String),
    SearchableField(name="Area", type=SearchFieldDataType.String),
    SearchableField(name="Block", type=SearchFieldDataType.Int32),
    SearchableField(name="Street", type=SearchFieldDataType.Int32),
    SearchableField(name="Building", type=SearchFieldDataType.Int32),
    SearchableField(name="Address", type=SearchFieldDataType.String),
    SearchableField(name="Gender", type=SearchFieldDataType.String, filterable=True),
    SearchField(name="fullName_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
    SearchField(name="address_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
    
]

vector_search = VectorSearch(
    algorithm_configurations=[
        VectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            kind="hnsw",
            hnsw_parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 500,
                "metric": "cosine"
            }
        )
    ]
)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        title_field=SemanticField(field_name="Name"),
        prioritized_keywords_fields=[SemanticField(field_name="CID")],
        prioritized_content_fields=[SemanticField(field_name="Address")]
    )
)

# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 citizens created


In [200]:
# Upload documents to the index
with open("./data/citizens-mock-data-vectors.json", 'r') as file:  
    documents = json.load(file)  
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(documents)  
print(f"Uploaded {len(documents)} documents") 

Uploaded 9 documents


## Perform Test Queries
We are performing a few test queries against the Cognitive Search index. If successful, it should display outdoor product information and images.

In [201]:
search_client = SearchClient(service_endpoint, index_name, credential=credential)  
fields_of_interest = ["CID", "Name", "Address", "DOB", "Nationality", "Gendor"]

In [202]:
# import requests
# from matplotlib import pyplot as plt
# from io import BytesIO

# def display_image_from_blob(image_file):
   
#   print(f"Image File: {image_file}")
#   print(f"blob_sas_url: {blob_sas_url}")
#   print(f"blob_sas_url.split()[0]: {blob_sas_url.split("?")[0]}")
#   #print(f"blob_sas_url.split()[1]: {blob_sas_url.split("?")[1]}")
  
#   # Append the image name to the SAS URL
#   image_url = blob_sas_url.split("?")[0] + f"/{image_file}?" # + blob_sas_url.split("?")[1]

#   # Get the image content
#   response = requests.get(image_url)

#   # Check if the request was successful
#   if response.status_code == 200:
#       # Open the image and display it
#       img = plt.imread(BytesIO(response.content))
#       plt.imshow(img)
#       plt.axis('off') # No axes for this plot
#       plt.show()
#   else:
#       print(f"Failed to retrieve image. HTTP Status code: {response.status_code}")

# def print_results(results):  
#   for result in results:  
#     print(f"Score: {result['@search.score']}")
#     print(f"Name: {result['name']}")  
#     print(f"Category: {result['category']}")
#     print(f"Tagline: {result['tagline']}")
#     print(f"Description: {result['description'][:50]}")
#     print(f"Original price: {result['original_price']}")
#     print(f"Special offer: {result['special_offer']}")
#     print(f"Image file: {result['product_image_file']}\n")
#     display_image_from_blob(result['product_image_file'])


# # Pure Vector Search with Filter
# query = "tent for two people"  
 
# results = search_client.search(  
#     search_text=None,  
#     vector=generate_embeddings(query), top_k=3,  
#     vector_fields="description_vector",
#     filter="category eq 'outdoor'",
#     select= fields_of_interest
# )  
  
# print_results(results)

## Create Azure SQL Database
Now we are creating a small Azure SQL Database with customer, products and order data using the SQL Server that you have deployed manually.

In [203]:
# Connection Strings
server_connection_string = f"Driver={{ODBC Driver 18 for SQL Server}};Server=tcp:{sql_db_server},1433;Uid={sql_db_user};Pwd={sql_db_password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
database_connection_string = server_connection_string + f"Database={sql_db_name};"

In [204]:
import random

citizens = [
    {"Name": "Ali Safar", "CID": "229897660", "Password" : "P@ssw0rd"},
    {"Name": "Mohammad Salem", "CID": "229897661", "Password" : "P@ssw0rd"},
    {"Name": "Abdulla Yousef", "CID": "229897662", "Password" : "P@ssw0rd"},
    {"Name": "Mariam Ibrahim", "CID": "229897663", "Password" : "P@ssw0rd"},
    {"Name": "Ali Alaa", "CID": "229897664", "Password" : "P@ssw0rd"},
    {"Name": "Ibrahim Ayoub", "CID": "229897665", "Password" : "P@ssw0rd"},
    {"Name": "Nour Ahmad", "CID": "229897666", "Password" : "P@ssw0rd"},
    {"Name": "Abrar Salem", "CID": "229897667", "Password" : "P@ssw0rd"},
    {"Name": "Younes Jawad", "CID": "229897668", "Password" : "P@ssw0rd"},
]

documents = [
    {"CID": "229897660", "Category": "Passport", "IssueDate": "7/14/2018", "ExpiryDate": "7/13/2023", "DocumentNumber": "H12P02"},
    {"CID": "229897661", "Category": "Passport", "IssueDate": "7/14/2019", "ExpiryDate": "7/13/2023", "DocumentNumber": "H12P03"},
    {"CID": "229897662", "Category": "Passport", "IssueDate": "7/14/2020", "ExpiryDate": "7/13/2023", "DocumentNumber": "H12P04"},
    {"CID": "229897663", "Category": "Passport", "IssueDate": "7/14/2021", "ExpiryDate": "7/13/2033", "DocumentNumber": "H12P05"},
    {"CID": "229897664", "Category": "Passport", "IssueDate": "7/14/2022", "ExpiryDate": "7/13/2033", "DocumentNumber": "H12P06"},
    {"CID": "229897665", "Category": "Passport", "IssueDate": "1/14/2023", "ExpiryDate": "7/13/2033", "DocumentNumber": "H12P07"},
    {"CID": "229897666", "Category": "Passport", "IssueDate": "2/14/2023", "ExpiryDate": "7/13/2033", "DocumentNumber": "H12P08"},
    {"CID": "229897667", "Category": "Passport", "IssueDate": "3/14/2023", "ExpiryDate": "7/13/2033", "DocumentNumber": "H12P09"},
    {"CID": "229897668", "Category": "Passport", "IssueDate": "4/14/2023", "ExpiryDate": "7/13/2033", "DocumentNumber": "H12P10"},

    {"CID": "229897660", "Category": "CivilID", "IssueDate": "7/14/2018", "ExpiryDate": "7/13/2023", "DocumentNumber": "229897660"},
    {"CID": "229897661", "Category": "CivilID", "IssueDate": "7/14/2019", "ExpiryDate": "7/13/2023", "DocumentNumber": "229897661"},
    {"CID": "229897662", "Category": "CivilID", "IssueDate": "7/14/2020", "ExpiryDate": "7/13/2023", "DocumentNumber": "229897662"},
    {"CID": "229897663", "Category": "CivilID", "IssueDate": "7/14/2021", "ExpiryDate": "7/13/2033", "DocumentNumber": "229897663"},
    {"CID": "229897664", "Category": "CivilID", "IssueDate": "7/14/2022", "ExpiryDate": "7/13/2033", "DocumentNumber": "229897664"},
    {"CID": "229897665", "Category": "CivilID", "IssueDate": "1/14/2023", "ExpiryDate": "7/13/2033", "DocumentNumber": "229897665"},
    {"CID": "229897666", "Category": "CivilID", "IssueDate": "2/14/2023", "ExpiryDate": "7/13/2033", "DocumentNumber": "229897666"},
    {"CID": "229897667", "Category": "CivilID", "IssueDate": "3/14/2023", "ExpiryDate": "7/13/2033", "DocumentNumber": "229897667"},
    {"CID": "229897668", "Category": "CivilID", "IssueDate": "4/14/2023", "ExpiryDate": "7/13/2033", "DocumentNumber": "229897668"},

    {"CID": "229897660", "Category": "Driving Licence", "IssueDate": "7/14/2018", "ExpiryDate": "7/13/2023", "DocumentNumber": "98098001"},
    {"CID": "229897661", "Category": "Driving Licence", "IssueDate": "7/14/2019", "ExpiryDate": "7/13/2023", "DocumentNumber": "98098002"},
    {"CID": "229897662", "Category": "Driving Licence", "IssueDate": "7/14/2020", "ExpiryDate": "7/13/2023", "DocumentNumber": "98098003"},
    {"CID": "229897663", "Category": "Driving Licence", "IssueDate": "7/14/2021", "ExpiryDate": "7/13/2033", "DocumentNumber": "98098004"},
    {"CID": "229897664", "Category": "Driving Licence", "IssueDate": "7/14/2022", "ExpiryDate": "7/13/2033", "DocumentNumber": "98098005"},
    {"CID": "229897665", "Category": "Driving Licence", "IssueDate": "1/14/2023", "ExpiryDate": "7/13/2033", "DocumentNumber": "98098006"},
    {"CID": "229897666", "Category": "Driving Licence", "IssueDate": "2/14/2023", "ExpiryDate": "7/13/2033", "DocumentNumber": "98098007"},
    {"CID": "229897667", "Category": "Driving Licence", "IssueDate": "3/14/2023", "ExpiryDate": "7/13/2033", "DocumentNumber": "98098008"},
    {"CID": "229897668", "Category": "Driving Licence", "IssueDate": "4/14/2023", "ExpiryDate": "7/13/2033", "DocumentNumber": "98098009"},
   
]
workPlaces = [
    {"CID": "229897660", "Company": "Microsoft", "HireDate": "3/22/2013", "Salary": random.randint(1000,50000)},
    {"CID": "229897661", "Company": "Ministry of Finance", "HireDate": "3/22/2014", "Salary": random.randint(1000,50000)},
    {"CID": "229897662", "Company": "Ministry of Interior", "HireDate": "3/22/2015", "Salary": random.randint(1000,50000)},
    {"CID": "229897663", "Company": "Ministry of Education", "HireDate": "3/22/2016", "Salary": random.randint(1000,50000)},
    {"CID": "229897664", "Company": "Retired", "HireDate": "3/22/2017", "Salary": random.randint(1000,50000)},
    {"CID": "229897665", "Company": "Minitry of Interior", "HireDate": "3/22/2018", "Salary": random.randint(1000,50000)},
    {"CID": "229897666", "Company": "Student", "HireDate": "3/22/2019", "Salary": random.randint(1000,50000)},
    {"CID": "229897667", "Company": "Retired", "HireDate": "3/22/2020", "Salary": random.randint(1000,50000)},
    {"CID": "229897668", "Company": "Minitry of Foriegn Affairs", "HireDate": "3/22/2021", "Salary": random.randint(1000,50000)},
]

In [205]:
# Test connection to the SQL Server

try:
    # Try to establish a connection
    conn = pyodbc.connect(server_connection_string)
    
    # If connection is successful, print a message and close the connection
    print("Connection to the server/database was successful!")
    conn.close()
    
except pyodbc.Error as ex:
    # Catch any connection errors and print them
    sqlstate = ex.args[0] if len(ex.args) > 0 else None
    message = ex.args[1] if len(ex.args) > 1 else None
    print(f"Failed to connect to the server/database. SQLSTATE: {sqlstate}, Message: {message}")

Connection to the server/database was successful!


In [206]:
# SET TO TRUE ONLY TO REBUILD DATABASE BASED ON ABOVE SAMPLE DATA
rebuild_database = True

if rebuild_database:

    # Connect to the server without specifying a database
    server_conn = pyodbc.connect(server_connection_string, autocommit=True)
    server_cursor = server_conn.cursor()

    # Drop the database if it exists
    server_cursor.execute(f"IF EXISTS(SELECT * FROM sys.databases WHERE name='{sql_db_name}') DROP DATABASE {sql_db_name}")

    # Recreate the database
    server_cursor.execute(f"CREATE DATABASE {sql_db_name}")
    server_cursor.close()
    server_conn.close()

    # Now, connect to the newly created database
    conn = pyodbc.connect(database_connection_string)
    cursor = conn.cursor()

    # Ensure you're using the existing database
    cursor.execute(f"USE {sql_db_name}")

    # Create tables and populate them
    cursor.execute("""
    CREATE TABLE Citizens (
        Name VARCHAR(255),
        CID varchar(12) PRIMARY KEY,
        Password varchar(20),
    )
    """)

    for ctizen in citizens:
        cursor.execute("INSERT INTO Citizens VALUES (?, ?, ?)", 
                    (ctizen["Name"], ctizen["CID"], ctizen["Password"]))

    cursor.execute("""
    CREATE TABLE Documents (
        CID varchar(12) ,
        Category VARCHAR(50),
        IssueDate DATE,
        ExpiryDate DATE,
        DocumentNumber VARCHAR(50),
        PRIMARY KEY (CID, Category)
    )
    """)

    for document in documents:
        cursor.execute("INSERT INTO Documents VALUES (?, ?, ?, ?, ?)", 
                    (document["CID"], document["Category"], document["IssueDate"], document["ExpiryDate"], document["DocumentNumber"]))

    cursor.execute("""
    CREATE TABLE WorkPlaces (
        CID varchar(12) PRIMARY KEY,
        Company VARCHAR(50),
        HireDate DATE,
        Salary DECIMAL(10,2)
    )
    """)

    for workPlace in workPlaces:
        cursor.execute("INSERT INTO WorkPlaces VALUES (?, ?, ?, ?)", 
                    (workPlace["CID"], workPlace["Company"], workPlace["HireDate"], workPlace["Salary"]))

    conn.commit()

    #Verify database tables and columns
    def fetch_schema_info():
        cursor.execute("""
            SELECT t.TABLE_NAME, c.COLUMN_NAME, c.DATA_TYPE 
            FROM INFORMATION_SCHEMA.TABLES AS t
            JOIN INFORMATION_SCHEMA.COLUMNS AS c ON t.TABLE_NAME = c.TABLE_NAME
            WHERE t.TABLE_SCHEMA = 'dbo'  -- assuming you're using the default schema
            ORDER BY t.TABLE_NAME, c.ORDINAL_POSITION
        """)
        
        tables = {}
        for row in cursor.fetchall():
            table_name = row[0]
            column_name = row[1]
            data_type = row[2]
            
            if table_name not in tables:
                tables[table_name] = []
            
            tables[table_name].append(f"{column_name} ({data_type})")
        
        return tables

    schema_info = fetch_schema_info()

    # Print the schema info in a user-friendly format
    for table, columns in schema_info.items():
        print(f"Table: {table}")
        for col in columns:
            print(f"    {col}")
        print()

    # Close connections
    cursor.close()
    conn.close()

DataError: ('22007', '[22007] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Conversion failed when converting date and/or time from character string. (241) (SQLExecDirectW)')